In [2]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


Using TensorFlow backend.


In [3]:
#Variable
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
#Character
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
#ctable.indices_char

In [7]:
print (ctable.chars)

[' ', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [8]:
#Data Generate
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))
print (questions[:5],expected[:5])

Generating data...
Total addition questions: 80000
['2+7    ', '4+970  ', '72+4   ', '9+9    ', '42+87  '] ['9   ', '974 ', '76  ', '18  ', '129 ']


In [9]:
#Processing
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)
input:  [[[False False False False False False False False False False False
    True]
  [False False False  True False False False False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]]

 [[False False False False False False False False False False False
    True]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False  True False False Fal

In [11]:
#Build Model
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Training
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 241us/step - loss: 2.0220 - acc: 0.2885 - val_loss: 1.8699 - val_acc: 0.3257
Q 105+73  T 178  ☒ 126 
Q 712+63  T 775  ☒ 126 
Q 79+865  T 944  ☒ 160 
Q 786+985 T 1771 ☒ 1106
Q 716+56  T 772  ☒ 126 
Q 148+36  T 184  ☒ 126 
Q 414+309 T 723  ☒ 126 
Q 695+109 T 804  ☒ 110 
Q 842+521 T 1363 ☒ 122 
Q 4+377   T 381  ☒ 12  

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 167us/step - loss: 1.8556 - acc: 0.3313 - val_loss: 1.8354 - val_acc: 0.3295
Q 947+925 T 1872 ☒ 1001
Q 751+835 T 1586 ☒ 100 
Q 657+0   T 657  ☒ 155 
Q 361+45  T 406  ☒ 135 
Q 301+8   T 309  ☒ 335 
Q 50+530  T 58

18000/18000 [==============================] - 3s 145us/step - loss: 1.2713 - acc: 0.5281 - val_loss: 1.2580 - val_acc: 0.5278
Q 69+743  T 812  ☒ 810 
Q 974+878 T 1852 ☒ 1849
Q 5+162   T 167  ☒ 161 
Q 62+74   T 136  ☒ 129 
Q 744+481 T 1225 ☒ 1290
Q 496+48  T 544  ☒ 543 
Q 10+56   T 66   ☒ 62  
Q 73+55   T 128  ☒ 129 
Q 538+33  T 571  ☒ 574 
Q 840+467 T 1307 ☒ 1329

--------------------------------------------------
Iteration 14
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 148us/step - loss: 1.2362 - acc: 0.5391 - val_loss: 1.2352 - val_acc: 0.5422
Q 55+257  T 312  ☒ 202 
Q 36+781  T 817  ☒ 819 
Q 344+74  T 418  ☒ 408 
Q 75+752  T 827  ☒ 810 
Q 533+49  T 582  ☒ 544 
Q 98+204  T 302  ☒ 300 
Q 91+57   T 148  ☒ 128 
Q 55+807  T 862  ☒ 736 
Q 463+177 T 640  ☒ 643 
Q 617+22  T 639  ☒ 638 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 148us/step - loss: 0.6552 - acc: 0.7606 - val_loss: 0.6620 - val_acc: 0.7451
Q 205+57  T 262  ☒ 263 
Q 47+179  T 226  ☒ 235 
Q 83+758  T 841  ☒ 839 
Q 322+28  T 350  ☒ 351 
Q 645+18  T 663  ☑ 663 
Q 833+250 T 1083 ☑ 1083
Q 70+521  T 591  ☒ 681 
Q 19+708  T 727  ☒ 726 
Q 217+24  T 241  ☑ 241 
Q 188+349 T 537  ☒ 546 

--------------------------------------------------
Iteration 28
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 141us/step - loss: 0.5926 - acc: 0.7883 - val_loss: 0.6162 - val_acc: 0.7734
Q 84+450  T 534  ☒ 535 
Q 24+794  T 818  ☒ 808 
Q 58+368  T 426  ☑ 426 
Q 875+595 T 1470 ☒ 1561
Q 256+7   T 263  ☒ 253 
Q 10+283  T 293  ☒ 202 
Q 461+327 T 788  ☒ 780 
Q 390+16  T 406  ☒ 308 
Q 85+511  T 596  ☒ 696 
Q 672+230 T 902  ☒ 903 

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 164us/step - loss: 0.1809 - acc: 0.9625 - val_loss: 0.2411 - val_acc: 0.9233
Q 82+653  T 735  ☑ 735 
Q 303+838 T 1141 ☒ 1131
Q 2+174   T 176  ☑ 176 
Q 40+80   T 120  ☑ 120 
Q 251+91  T 342  ☒ 332 
Q 128+752 T 880  ☒ 881 
Q 526+244 T 770  ☑ 770 
Q 219+391 T 610  ☑ 610 
Q 355+14  T 369  ☒ 379 
Q 43+82   T 125  ☑ 125 

--------------------------------------------------
Iteration 42
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 158us/step - loss: 0.1668 - acc: 0.9663 - val_loss: 0.2083 - val_acc: 0.9414
Q 214+79  T 293  ☑ 293 
Q 1+225   T 226  ☑ 226 
Q 136+23  T 159  ☒ 160 
Q 578+830 T 1408 ☑ 1408
Q 141+370 T 511  ☑ 511 
Q 715+52  T 767  ☑ 767 
Q 26+438  T 464  ☑ 464 
Q 322+980 T 1302 ☑ 1302
Q 57+302  T 359  ☒ 350 
Q 964+143 T 1107 ☒ 1007

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 154us/step - loss: 0.0630 - acc: 0.9906 - val_loss: 0.1128 - val_acc: 0.9649
Q 966+136 T 1102 ☒ 1002
Q 996+10  T 1006 ☑ 1006
Q 607+429 T 1036 ☑ 1036
Q 342+26  T 368  ☑ 368 
Q 352+631 T 983  ☑ 983 
Q 374+79  T 453  ☑ 453 
Q 867+196 T 1063 ☒ 1053
Q 785+743 T 1528 ☑ 1528
Q 22+681  T 703  ☑ 703 
Q 78+791  T 869  ☒ 879 

--------------------------------------------------
Iteration 56
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 147us/step - loss: 0.0763 - acc: 0.9830 - val_loss: 0.1463 - val_acc: 0.9525
Q 259+136 T 395  ☑ 395 
Q 612+13  T 625  ☑ 625 
Q 51+281  T 332  ☑ 332 
Q 268+923 T 1191 ☑ 1191
Q 759+274 T 1033 ☑ 1033
Q 83+730  T 813  ☑ 813 
Q 360+796 T 1156 ☑ 1156
Q 26+144  T 170  ☒ 160 
Q 802+383 T 1185 ☒ 1184
Q 405+46  T 451  ☑ 451 

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 2s 139us/step - loss: 0.0315 - acc: 0.9962 - val_loss: 0.0734 - val_acc: 0.9775
Q 796+219 T 1015 ☑ 1015
Q 98+210  T 308  ☑ 308 
Q 180+48  T 228  ☑ 228 
Q 578+4   T 582  ☑ 582 
Q 908+90  T 998  ☑ 998 
Q 464+683 T 1147 ☑ 1147
Q 34+472  T 506  ☑ 506 
Q 959+30  T 989  ☑ 989 
Q 374+473 T 847  ☑ 847 
Q 481+184 T 665  ☑ 665 

--------------------------------------------------
Iteration 70
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 140us/step - loss: 0.0233 - acc: 0.9985 - val_loss: 0.0661 - val_acc: 0.9798
Q 188+51  T 239  ☑ 239 
Q 542+115 T 657  ☑ 657 
Q 74+406  T 480  ☑ 480 
Q 374+473 T 847  ☑ 847 
Q 99+547  T 646  ☑ 646 
Q 98+538  T 636  ☑ 636 
Q 978+28  T 1006 ☑ 1006
Q 514+1   T 515  ☑ 515 
Q 964+33  T 997  ☑ 997 
Q 16+996  T 1012 ☒ 1011

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 2s 139us/step - loss: 0.0123 - acc: 0.9992 - val_loss: 0.0587 - val_acc: 0.9810
Q 613+5   T 618  ☑ 618 
Q 944+353 T 1297 ☑ 1297
Q 64+856  T 920  ☑ 920 
Q 78+426  T 504  ☑ 504 
Q 805+550 T 1355 ☑ 1355
Q 33+325  T 358  ☑ 358 
Q 163+459 T 622  ☑ 622 
Q 213+850 T 1063 ☒ 1062
Q 866+943 T 1809 ☒ 1819
Q 8+262   T 270  ☑ 270 

--------------------------------------------------
Iteration 84
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 143us/step - loss: 0.0123 - acc: 0.9991 - val_loss: 0.0572 - val_acc: 0.9806
Q 310+29  T 339  ☑ 339 
Q 57+677  T 734  ☑ 734 
Q 2+379   T 381  ☑ 381 
Q 0+793   T 793  ☑ 793 
Q 33+709  T 742  ☑ 742 
Q 103+87  T 190  ☒ 180 
Q 572+88  T 660  ☑ 660 
Q 879+372 T 1251 ☑ 1251
Q 783+784 T 1567 ☑ 1567
Q 38+261  T 299  ☑ 299 

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 144us/step - loss: 0.1323 - acc: 0.9568 - val_loss: 0.1722 - val_acc: 0.9368
Q 554+649 T 1203 ☒ 1202
Q 86+185  T 271  ☑ 271 
Q 790+75  T 865  ☑ 865 
Q 848+907 T 1755 ☑ 1755
Q 802+1   T 803  ☑ 803 
Q 153+743 T 896  ☒ 996 
Q 177+690 T 867  ☑ 867 
Q 636+203 T 839  ☒ 849 
Q 228+83  T 311  ☑ 311 
Q 605+58  T 663  ☑ 663 

--------------------------------------------------
Iteration 98
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 138us/step - loss: 0.0667 - acc: 0.9793 - val_loss: 0.0649 - val_acc: 0.9771
Q 5+27    T 32   ☑ 32  
Q 146+900 T 1046 ☒ 1055
Q 721+486 T 1207 ☑ 1207
Q 98+178  T 276  ☑ 276 
Q 877+569 T 1446 ☑ 1446
Q 69+223  T 292  ☑ 292 
Q 13+188  T 201  ☑ 201 
Q 756+339 T 1095 ☑ 1095
Q 84+808  T 892  ☑ 892 
Q 38+261  T 299  ☑ 299 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

In [13]:
#Testing
print("MSG : Prediction")
test_x = ["555+275", "860+7  ", "340+29 "]
test_y = ["830 ", "867 ", "369 "]
x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
for j, (i, c) in enumerate(zip(test_x, test_y)):
    x[j] = ctable.encode(i, MAXLEN)
    y[j] = ctable.encode(c, DIGITS + 1)

MSG : Prediction


In [14]:
right = 0
preds = model.predict_classes(x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(x[i])
    correct = ctable.decode(y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

Q 555+275 T 830  ☑ 830 
Q 860+7   T 867  ☑ 867 
Q 340+29  T 369  ☑ 369 
MSG : Accuracy is 1.0
